In [1]:
class FinancialDataset:

    def __init__(self, driving_csv, target_csv, window_size, split_ratio=0.8, normalize=False):
        driving_data = pd.read_csv(driving_csv)
        target_data = pd.read_csv(target_csv)

        driving_data = driving_data['Close'].fillna(method='pad')
        target_data = target_data['Close'].fillna(method='pad')
        self.train_samples = int(split_ratio * (target_data.shape[0] - window_size - 1))
        self.test_samples = target_data.shape[0] - window_size - 1 - self.train_samples
        self.mean_value = target_data.mean()
        if normalize:
            target_data = target_data - target_data.mean()

        self.features, self.targets, self.target_sequences = self.generate_time_series(driving_data, target_data,window_size)

    def get_sample_sizes(self):
        return self.train_samples, self.test_samples

    def get_feature_count(self):
        return self.features.shape[1]

    def get_training_data(self):
        return self.features[:self.train_samples], self.targets[:self.train_samples], self.target_sequences[:self.train_samples]

    def get_testing_data(self):
        return self.features[self.train_samples:], self.targets[self.train_samples:], self.target_sequences[self.train_samples:]

    def generate_time_series(self, driving_data, target_data, window_size):
        feature_list, target_list, target_sequence_list = [], [], []
        for i in range(len(driving_data) - window_size - 1):
            end_idx = i + window_size
            feature_list.append(driving_data[i: end_idx])
            target_list.append(target_data[end_idx])
            target_sequence_list.append(target_data[i: end_idx])
        return np.array(feature_list), np.array(target_list), np.array(target_sequence_list)


import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F


class AttentionEncoder(nn.Module):

    def __init__(self, input_dim, hidden_dim, sequence_length, dropout_rate):
        super(AttentionEncoder, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.sequence_length = sequence_length

        self.lstm_layer = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=1)
        self.attn_layer1 = nn.Linear(in_features=2 * hidden_dim, out_features=self.sequence_length)
        self.attn_layer2 = nn.Linear(in_features=self.sequence_length, out_features=self.sequence_length)
        self.tanh_activation = nn.Tanh()
        self.attn_layer3 = nn.Linear(in_features=self.sequence_length, out_features=1)

        self.dropout_layer = nn.Dropout(dropout_rate)

    def forward(self, driving_input):
        batch_size = driving_input.size(0)
        encoded_output = self.initialize_variable(batch_size, self.sequence_length, self.hidden_dim)
        hidden_state = self.initialize_variable(1, batch_size, self.hidden_dim)
        cell_state = self.initialize_variable(1, batch_size, self.hidden_dim)
        for t in range(self.sequence_length):
            combined_hidden = torch.cat((self.repeat_hidden(hidden_state), self.repeat_hidden(cell_state)), 2)
            attn_scores1 = self.attn_layer1(combined_hidden)
            attn_scores2 = self.attn_layer2(driving_input.permute(0, 2, 1))
            attn_combined = attn_scores1 + attn_scores2
            attn_weights = self.attn_layer3(self.tanh_activation(attn_combined))
            if batch_size > 1:
                attention_weights = F.softmax(attn_weights.view(batch_size, self.input_dim), dim=1)
            else:
                attention_weights = self.initialize_variable(batch_size, self.input_dim) + 1
            weighted_input = torch.mul(attention_weights, driving_input[:, t, :])

            weighted_input = self.dropout_layer(weighted_input)

            _, (hidden_state, cell_state) = self.lstm_layer(weighted_input.unsqueeze(0), (hidden_state, cell_state))
            encoded_output[:, t, :] = hidden_state

        return encoded_output

    def initialize_variable(self, *args):
        zero_tensor = torch.zeros(args)
        if torch.cuda.is_available():
            zero_tensor = zero_tensor.cuda()
        return Variable(zero_tensor)

    def repeat_hidden(self, hidden_tensor):
        return hidden_tensor.repeat(self.input_dim, 1, 1).permute(1, 0, 2)


class AttentionDecoder(nn.Module):

    def __init__(self, encoded_hidden_dim, hidden_dim, sequence_length, dropout_rate):
        super(AttentionDecoder, self).__init__()
        self.encoded_hidden_dim = encoded_hidden_dim
        self.hidden_dim = hidden_dim
        self.sequence_length = sequence_length

        self.attn_layer1 = nn.Linear(in_features=2 * hidden_dim, out_features=encoded_hidden_dim)
        self.attn_layer2 = nn.Linear(in_features=encoded_hidden_dim, out_features=encoded_hidden_dim)
        self.tanh_activation = nn.Tanh()
        self.attn_layer3 = nn.Linear(in_features=encoded_hidden_dim, out_features=1)
        self.lstm_layer = nn.LSTM(input_size=1, hidden_size=self.hidden_dim)
        self.concat_layer = nn.Linear(in_features=self.encoded_hidden_dim + 1, out_features=1)
        self.fc_layer1 = nn.Linear(in_features=encoded_hidden_dim + hidden_dim, out_features=hidden_dim)
        self.fc_layer2 = nn.Linear(in_features=hidden_dim, out_features=1)

        self.dropout_layer = nn.Dropout(dropout_rate)

    def forward(self, encoded_hidden, target_sequence):
        batch_size = encoded_hidden.size(0)
        decoder_hidden = self.initialize_variable(1, batch_size, self.hidden_dim)
        cell_state = self.initialize_variable(1, batch_size, self.hidden_dim)
        context_vector = self.initialize_variable(batch_size, self.hidden_dim)

        for t in range(self.sequence_length):
            combined_hidden = torch.cat((self.repeat_hidden(decoder_hidden), self.repeat_hidden(cell_state)), 2)
            attn_scores1 = self.attn_layer1(combined_hidden)
            attn_scores2 = self.attn_layer2(encoded_hidden)
            attn_combined = attn_scores1 + attn_scores2
            attn_weights = self.attn_layer3(self.tanh_activation(attn_combined))
            if batch_size > 1:
                beta_t = F.softmax(attn_weights.view(batch_size, -1), dim=1)
            else:
                beta_t = self.initialize_variable(batch_size, self.encoded_hidden_dim) + 1
            context_vector = torch.bmm(beta_t.unsqueeze(1), encoded_hidden).squeeze(1)
            if t < self.sequence_length - 1:
                concat_input = torch.cat((target_sequence[:, t].unsqueeze(1), context_vector), dim=1)
                y_tilde = self.concat_layer(concat_input)

                y_tilde = self.dropout_layer(y_tilde)

                _, (decoder_hidden, cell_state) = self.lstm_layer(y_tilde.unsqueeze(0), (decoder_hidden, cell_state))
        output = self.fc_layer2(
            self.dropout_layer(self.fc_layer1(torch.cat((decoder_hidden.squeeze(0), context_vector), dim=1))))

        return output

    def initialize_variable(self, *args):
        zero_tensor = torch.zeros(args)
        if torch.cuda.is_available():
            zero_tensor = zero_tensor.cuda()
        return Variable(zero_tensor)

    def repeat_hidden(self, hidden_tensor):
        return hidden_tensor.repeat(self.sequence_length, 1, 1).permute(1, 0, 2)


import torch
from torch import nn, optim
from torch.autograd import Variable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from torch.optim.lr_scheduler import ReduceLROnPlateau


class FinancialModelTrainer:

    def __init__(self, driving_file, target_file, sequence_length, split_fraction, learning_rate, dropout_rate,
                 encoder_hidden_dim, decoder_hidden_dim):
        self.dataset = FinancialDataset(driving_file, target_file, sequence_length, split_fraction, normalize=True)
        self.encoder = AttentionEncoder(input_dim=self.dataset.get_feature_count(), hidden_dim=encoder_hidden_dim,
                                        sequence_length=sequence_length, dropout_rate=dropout_rate)
        self.decoder = AttentionDecoder(encoded_hidden_dim=encoder_hidden_dim, hidden_dim=decoder_hidden_dim,
                                        sequence_length=sequence_length, dropout_rate=dropout_rate)
        if torch.cuda.is_available():
            self.encoder = self.encoder.cuda()
            self.decoder = self.decoder.cuda()
        self.encoder_optimizer = optim.Adam(self.encoder.parameters(), learning_rate, weight_decay=1e-5)
        self.decoder_optimizer = optim.Adam(self.decoder.parameters(), learning_rate, weight_decay=1e-5)

        self.encoder_scheduler = ReduceLROnPlateau(self.encoder_optimizer, mode='min', factor=0.5, patience=1,
                                                   verbose=False, threshold=0.0001, threshold_mode='rel', cooldown=0,
                                                   min_lr=0, eps=1e-08)
        self.decoder_scheduler = ReduceLROnPlateau(self.decoder_optimizer, mode='min', factor=0.5, patience=1,
                                                   verbose=False, threshold=0.0001, threshold_mode='rel', cooldown=0,
                                                   min_lr=0, eps=1e-08)

        self.loss_function = nn.MSELoss()
        self.train_samples, self.test_samples = self.dataset.get_sample_sizes()

    def train_model(self, num_epochs, batch_size, save_interval):
        x_train, y_train, y_seq_train = self.dataset.get_training_data()
        for epoch in range(num_epochs):
            i = 0
            loss_sum = 0
            while (i < self.train_samples):
                self.encoder_optimizer.zero_grad()
                self.decoder_optimizer.zero_grad()
                batch_end = i + batch_size
                if (batch_end >= self.train_samples):
                    batch_end = self.train_samples
                var_x = self.to_variable(x_train[i: batch_end])
                var_y = self.to_variable(y_train[i: batch_end])
                var_y_seq = self.to_variable(y_seq_train[i: batch_end])
                if var_x.dim() == 2:
                    var_x = var_x.unsqueeze(2)
                encoded_sequence = self.encoder(var_x)
                y_pred = self.decoder(encoded_sequence, var_y_seq)
                loss = self.loss_function(y_pred, var_y)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.encoder.parameters(), max_norm=1.0)
                torch.nn.utils.clip_grad_norm_(self.decoder.parameters(), max_norm=1.0)
                self.encoder_optimizer.step()
                self.decoder_optimizer.step()
                loss_sum += loss.item()
                i = batch_end
            print('epoch [%d] ==> loss is %f' % (epoch, loss_sum / self.train_samples))

            self.encoder_scheduler.step(loss_sum)
            self.decoder_scheduler.step(loss_sum)

            if (epoch + 1) % save_interval == 0 or epoch + 1 == num_epochs:
                self.save_model(epoch + 1)

    def test_model(self, num_epochs, batch_size,isPlot):
        x_train, y_train, y_seq_train = self.dataset.get_training_data()
        x_test, y_test, y_seq_test = self.dataset.get_testing_data()
        y_pred_train = self.predict(x_train, y_train, y_seq_train, batch_size)
        y_pred_test = self.predict(x_test, y_test, y_seq_test, batch_size)
        if isPlot:
            plt.figure(figsize=(8, 6), dpi=100)
            plt.plot(range(0, self.train_samples), y_train[:], label='train actual', color='green')
            plt.plot(range(self.train_samples, self.train_samples + self.test_samples), y_test, label='test actual',
                     color='black')
            plt.plot(range(0, self.train_samples), y_pred_train[:], label='predicted train', color='red')
            plt.plot(range(self.train_samples, self.train_samples + self.test_samples), y_pred_test, label='predicted test',
                     color='blue')
            plt.xlabel('Days')
            plt.ylabel('Price')
            plt.legend()
            plt.show()
        return y_pred_test + self.dataset.mean_value, y_test + self.dataset.mean_value

    def predict(self, x_data, y_data, y_seq_data, batch_size):
        y_pred = np.zeros(x_data.shape[0])
        i = 0
        while (i < x_data.shape[0]):
            batch_end = i + batch_size
            if batch_end > x_data.shape[0]:
                batch_end = x_data.shape[0]
            var_x_input = self.to_variable(x_data[i: batch_end])
            var_y_input = self.to_variable(y_seq_data[i: batch_end])
            if var_x_input.dim() == 2:
                var_x_input = var_x_input.unsqueeze(2)
            encoded_sequence = self.encoder(var_x_input)
            y_res = self.decoder(encoded_sequence, var_y_input)
            for j in range(i, batch_end):
                y_pred[j] = y_res[j - i, -1]
            i = batch_end
        return y_pred

    def load_model(self, encoder_path, decoder_path):
        self.encoder.load_state_dict(torch.load(encoder_path, map_location=lambda storage, loc: storage))
        self.decoder.load_state_dict(torch.load(decoder_path, map_location=lambda storage, loc: storage))

    def save_model(self, epoch):
        if not os.path.exists('models'):
            os.makedirs('models')
        encoder_path = f'models/encoder.model'
        decoder_path = f'models/decoder.model'
        torch.save(self.encoder.state_dict(), encoder_path)
        torch.save(self.decoder.state_dict(), decoder_path)

    def to_variable(self, x):
        if torch.cuda.is_available():
            return Variable(torch.from_numpy(x).float()).cuda()
        else:
            return Variable(torch.from_numpy(x).float())


import yfinance as yf

num_epochs = 30
batch_size = 64
split_fraction = 0.8
save_interval = 99
learning_rate = 0.005
sequence_length = 10
encoder_hidden_dim = 64
decoder_hidden_dim = 64
driving_data_filename = 'driver.csv'
target_data_filename = 'target.csv'
dropout_rate = 0.01
predictions = []
true_values = []
lag_days = 5
counter = 0

trainer = None
while counter <= 17:
    driver_data = yf.download('^GSPC', start=str(2002 + counter) + '-01-01', end=str(2006 + counter) + '-12-31').iloc[
                  :-lag_days, :]
    target_data = yf.download('^GSPC', start=str(2002 + counter) + '-01-01', end=str(2006 + counter) + '-12-31').iloc[
                  lag_days:, :]

    driver_data.to_csv(driving_data_filename)
    target_data.to_csv(target_data_filename)

    trainer = FinancialModelTrainer(driving_data_filename, target_data_filename, sequence_length, split_fraction,
                                    learning_rate, dropout_rate, encoder_hidden_dim, decoder_hidden_dim)
    trainer.train_model(num_epochs, batch_size, save_interval)
    year_pred, year_true = trainer.test_model(num_epochs, batch_size,isPlot = False)

    predictions += list(year_pred)
    true_values += list(year_true)

    counter += 1

predictions_df = pd.DataFrame(predictions)
true_values_df = pd.DataFrame(true_values)
predictions_df.to_csv('Predictions.csv')
true_values_df.to_csv('Correct.csv')


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['^GSPC']: ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'fc.yahoo.com\', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001FA2F683E00>: Failed to resolve \'fc.yahoo.com\' ([Errno 11001] getaddrinfo failed)"))'))
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['^GSPC']: ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'fc.yahoo.com\', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001FA3129AAE0>: Failed to resolve \'fc.yahoo.com\' ([Errno 11001] getaddrinfo failed)"))'))
C:\Users\rutwi\AppData\Local\Temp\ipykernel_19796\2350723139.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  driving_data

IndexError: tuple index out of range

In [ ]:
import pandas as pd

predictions_df = pd.read_csv('Predictions.csv')
threshold_pct_change = 0.05
predictions_df['Signal'] = 0
predictions_df['Pct_Change'] = predictions_df['0'].pct_change()
predictions_df.loc[predictions_df['Pct_Change'] > threshold_pct_change, 'Signal'] = 1
predictions_df.loc[predictions_df['Pct_Change'] < -threshold_pct_change, 'Signal'] = -1


predictions_df.drop(columns=['Pct_Change'], inplace=True)


predictions_df.to_csv('Predictions_with_Signals.csv', index=False)


print(predictions_df.head())


In [ ]:
combined_df["pct_diff"] = combined_df["Predicted Prices"]/ combined_df["Close"] - 1
combined_df["1pct_signal"] = combined_df["pct_diff"].apply(lambda x: 1 if x > 0.01 else -1 if x < -0.01 else 0)
combined_df["2pct_signal"] = combined_df["pct_diff"].apply(lambda x: 1 if x > 0.02 else -1 if x < -0.02 else 0)
combined_df["3pct_signal"] = combined_df["pct_diff"].apply(lambda x: 1 if x > 0.03 else -1 if x < -0.03 else 0)
combined_df["5pct_signal"] = combined_df["pct_diff"].apply(lambda x: 1 if x > 0.05 else -1 if x < -0.05 else 0)
combined_df["7pct_signal"] = combined_df["pct_diff"].apply(lambda x: 1 if x > 0.07 else -1 if x < -0.07 else 0)
combined_df["10pct_signal"] = combined_df["pct_diff"].apply(lambda x: 1 if x > 0.1 else -1 if x < -0.1 else 0)
combined_df["Asym_5pct_15pct_signal"] = combined_df["pct_diff"].apply(lambda x: 1 if x > 0.05 else -1 if x < -0.15 else 0)
combined_df["Asym_5pct_20pct_signal"] = combined_df["pct_diff"].apply(lambda x: 1 if x > 0.05 else -1 if x < -0.2 else 0)